In [1]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import scale

In [21]:
# Define working folder
root_folder = r"D:\Software\temp\CeDX\data" # <========= CHANGE THIS LINE TO ROOT FOLDER of data
tmp1_folder = root_folder + "/data/tmp"
tmp2_folder = root_folder + "/data/tmp2"
input_folder = root_folder + "/input"
output_folder = root_folder + "/output"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
year_limit = 2023

In [31]:
# Read processed service data from code 1
service = pd.read_csv(root_folder + '/data/service_data.csv')
service['DOC_DATE'] = pd.to_datetime(service['DOC_DATE'], format='%Y%m%d', errors='coerce')
service['DROP_OFF_DATE'] = pd.to_datetime(service['DROP_OFF_DATE'], format='%Y%m%d', errors='coerce')
service['PICK_UP_DATE'] = pd.to_datetime(service['PICK_UP_DATE'], format='%Y%m%d', errors='coerce')
service['DOC_year'] = service['DOC_DATE'].dt.year.fillna(0)
service['GROSS_VALUE'] = service['GROSS_VALUE'].apply(lambda x: 0 if x < 0 else float(x))
service = service[(service['DOC_year'] > 2015) & (service['DOC_year'] <= year_limit)]
#service = service.drop(['customer_index', 'Cust_group'], axis=1)

C:\Users\AO_HVN_ITS_LABO_DWH\AppData\Local\Temp\ipykernel_11732\2821949603.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  service = pd.read_csv(root_folder + '/data/service_data.csv')


In [29]:
# Read processed sales data from code 2
sale = pd.read_csv(input_folder + '/sale_uni_min.csv')
sale['MODEL'] = sale['MODEL_TEXT_1'].str.split(' ').str[0]
sale = sale.drop('MODEL_TEXT_1', axis=1)
sale['WARRANTY_START_DATE'] = pd.to_datetime(sale['WARRANTY_START_DATE'], format='%Y-%m-%d')
sale['START_year'] = sale['WARRANTY_START_DATE'].dt.year
sale = sale[(sale['START_year'] > 2015) & (sale['START_year'] <= year_limit)]
# sale = sale.drop(['DEALER_CODE', 'customer_index', 'Cust_group'], axis=1)
sale = sale.drop(['DEALER_CODE'], axis=1)

C:\Users\AO_HVN_ITS_LABO_DWH\AppData\Local\Temp\ipykernel_11732\3623793725.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  sale = pd.read_csv(input_folder + '/sale_uni_min.csv')


In [34]:
# Join sale and service data based on UID, left join
#service = service.rename(columns={'VIN' : 'UID'})
cb = pd.merge(sale, right=service, how='left', left_on= 'VIN',right_on='VIN')

# Calculate number of years between DOC_DATE and WARRANTY_START_DATE
cb['keika'] = np.ceil((cb['DOC_DATE'] - cb['WARRANTY_START_DATE']).dt.days / 365)

# ?
#cb['keika'] = cb['keika'].replace([np.inf, -np.inf], np.nan)
cb['keika'] = cb['keika'].apply(lambda x: 1 if x == 0 else float(x))
# Warranty = JOB_Type 07
cb['bad_q'] = cb['JOB_TYPE'].apply(lambda x: 0 if pd.isna(x) else (1 if x == 7 or str(x) == '07' or str(x) == '7' else 0))
cb = cb[cb['keika'] > 0]

del sale, service
print(cb.head())

   CUSTOMER_CODE_x  PARTNER_CATEGORY  TITLE CUSTOMER_NAME GENDER          UID  \
0       2000000000                 1    2.0   Lê Anh Quân   Male  913234781.0   
1       2000000000                 1    2.0   Lê Anh Quân   Male  913234781.0   
2       2000000000                 1    2.0   Lê Anh Quân   Male  913234781.0   
3       2000000000                 1    2.0   Lê Anh Quân   Male  913234781.0   
4       2000000000                 1    2.0   Lê Anh Quân   Male  913234781.0   

   DATEOFBIRTH OCCUPATION  REGION    CITY  ... GROSS_VALUE ORDER_STATUS  \
0          NaN        NaN       1  Hà Nội  ...     50600.0         H060   
1          NaN        NaN       1  Hà Nội  ...   1741344.0         H060   
2          NaN        NaN       1  Hà Nội  ...         0.0         H060   
3          NaN        NaN       1  Hà Nội  ...    303600.0         H070   
4          NaN        NaN       1  Hà Nội  ...    506000.0         H070   

  CUSTOMER_ADVISER  JOB  JOB_TYPE JOB_TYPE_DESCRIPTION  \
0   

In [35]:
cb.to_csv(output_folder + '/merged_sale_service.csv')

In [36]:
# Calculate tgt v2
tgt = cb[cb['JOB_TYPE_DESCRIPTION'] == 'Periodic Maintenance']
tgt = tgt.groupby('UID').agg({'DOC_DATE': lambda x: len(x.unique()), 'keika': 'max'}).reset_index()
tgt.columns = ['UID', 'PM', 'keika_max']
tgt['freq'] = tgt['PM'] / tgt['keika_max']
tgt = tgt[tgt['freq'] < 11]
tgt['PM_num'] = np.floor(tgt['freq'])
tgt['PM2ov'] = tgt['PM_num'].apply(lambda x: 1 if x >= 2 else 0)
tgt = tgt[['UID', 'PM2ov']]
print(tgt.head())

          UID  PM2ov
0         0.0      0
1  10920827.0      0
2  15768909.0      1
3  70209999.0      1
4  76716756.0      0


In [37]:
# Calculate SA_cont
cb = cb[cb['DOC_year'] < year_limit]
SA_cont = cb[['UID', 'DOC_DATE', 'CUSTOMER_ADVISER']].drop_duplicates()
SA_cont['rank'] = SA_cont.groupby('UID')['DOC_DATE'].rank()
max_rank = SA_cont.groupby('UID')['rank'].transform('max')
SA_cont = SA_cont[SA_cont['rank'] > max_rank - 3]
SA_cont['n'] = SA_cont.groupby('UID')['UID'].transform('count')
SA_cont = SA_cont.groupby(['UID', 'CUSTOMER_ADVISER']).agg({'n': 'count'}).reset_index()
SA_cont.columns = ['UID', 'CUSTOMER_ADVISER', 'count']
SA_cont = SA_cont[SA_cont['count'] > 2]
SA_cont = SA_cont.drop('count', axis=1)
SA_cont['SA_cont'] = 1
print(SA_cont.head())

            UID  CUSTOMER_ADVISER  SA_cont
0           0.0        20405002.0        1
24  112259319.0        10405020.0        1
48  128757808.0        21005014.0        1
56  151959200.0        10805017.0        1
63  164333198.0        20705016.0        1


In [38]:
# Calculate JOB_m
JOB_m = cb[cb['JOB_TYPE_DESCRIPTION'].isin(['Body', 'Paint', 'General Repair', 'Periodic Maintenance'])]
JOB_m = JOB_m.groupby(['UID', 'JOB_TYPE_DESCRIPTION']).agg({'GROSS_VALUE': 'sum'}).reset_index()
JOB_m.columns = ['UID', 'JOB_TYPE_DESCRIPTION', 'Total_VALUE']
JOB_m = JOB_m.pivot(index='UID', columns='JOB_TYPE_DESCRIPTION', values='Total_VALUE').reset_index()
JOB_m = JOB_m.fillna(0)
JOB_m['BP'] = JOB_m['Paint'] + JOB_m['Body']
JOB_m = JOB_m.drop(['Paint', 'Body'], axis=1)
JOB_m = JOB_m.rename(columns={'General Repair': 'GR', 'Periodic Maintenance': 'PM'})
print(JOB_m.head())

JOB_TYPE_DESCRIPTION         UID         GR         PM         BP
0                            0.0        0.0    11000.0        0.0
1                     15768909.0        0.0    55000.0   110000.0
2                     70209999.0        0.0  2085840.0        0.0
3                     76716756.0  1188220.0        0.0        0.0
4                     90492647.0  1155000.0  2998166.0  7986000.0


In [39]:
# Calculate JOB_vnum
JOB_vnum = cb[cb['JOB_TYPE_DESCRIPTION'].isin(['Body', 'Paint', 'General Repair', 'Periodic Maintenance'])]
JOB_vnum = JOB_vnum.groupby(['UID', 'JOB_TYPE_DESCRIPTION']).agg({'DOC_DATE': lambda x: len(x.unique())}).reset_index()
JOB_vnum.columns = ['UID', 'JOB_TYPE_DESCRIPTION', 'visit_num']
JOB_vnum = JOB_vnum.pivot(index='UID', columns='JOB_TYPE_DESCRIPTION', values='visit_num').reset_index()
JOB_vnum = JOB_vnum.fillna(0)
JOB_vnum['BP_vnum'] = JOB_vnum['Paint'] + JOB_vnum['Body']
JOB_vnum = JOB_vnum.drop(['Paint', 'Body'], axis=1)
JOB_vnum = JOB_vnum.rename(columns={'General Repair': 'GR_vnum', 'Periodic Maintenance': 'PM_vnum'})
print(JOB_vnum.head())

JOB_TYPE_DESCRIPTION         UID  GR_vnum  PM_vnum  BP_vnum
0                            0.0      1.0      1.0      1.0
1                     15768909.0      0.0      1.0      1.0
2                     70209999.0      0.0      2.0      0.0
3                     76716756.0      1.0      0.0      0.0
4                     90492647.0      2.0      2.0      1.0


In [40]:
# Calculate TB
TB = cb.copy()
TB['TIRE'] = TB['DESCRIPTION_ONE'].str.contains('TIRE').astype(int)
TB['BATTERY'] = TB['DESCRIPTION_ONE'].str.contains('BATTERY').astype(int)
TB['OIL'] = ((TB['DESCRIPTION_ONE'].str.contains('OIL')) & (TB['DESCRIPTION_ONE'].str.contains('HONDA'))).astype(int)
TB = TB.groupby('UID').agg({'TIRE': 'sum', 'BATTERY': 'sum', 'OIL': 'sum'}).reset_index()
print(TB.head())

          UID  TIRE  BATTERY  OIL
0         0.0     0        0    0
1  15768909.0     0        0    0
2  70209999.0     0        0    0
3  76716756.0     0        0    1
4  90492647.0     0        0    2


In [41]:
# Calculate PM
PM = cb[cb['JOB_TYPE_DESCRIPTION'] == 'Periodic Maintenance']
PM = PM.groupby('UID').agg({'DOC_year': ['max', lambda x: len(x.unique())], 'START_year': 'min'}).reset_index()
PM.columns = ['UID', 'latest_visit_year', 'visit_year', 'START_year']
PM['visit_rate'] = PM['visit_year'] / (year_limit - PM['START_year'])
PM = PM.drop('START_year', axis=1)
print(PM.head())

           UID  latest_visit_year  visit_year  visit_rate
0          0.0             2022.0           1    0.142857
1   15768909.0             2022.0           1    1.000000
2   70209999.0             2022.0           1    1.000000
3   90492647.0             2022.0           2    0.500000
4  108059030.0             2022.0           2    0.666667


In [42]:
# Calculate smr
smr = cb.groupby('UID').agg({'COUNTER_READING': 'max', 'GROSS_VALUE': 'sum', 'SALES_PRICE': 'sum', 'DOC_DATE': lambda x: len(x.unique()), 'keika': 'max', 'bad_q': 'max', 'START_year': 'min', 'WARRANTY_START_DATE': 'min', 'total_buy': 'max'}).reset_index()
smr.columns = ['UID', 'RUN', 'Total_VALUE', 'SALES_PRICE', 'visit_num', 'keika_max', 'bad_q', 'START_year', 'START_date', 'total_buy']
smr['Total_VALUE'] = smr['Total_VALUE'] + smr['SALES_PRICE']
smr = smr.drop('SALES_PRICE', axis=1)
smr['visit_freq_y'] = smr['visit_num'] / smr['keika_max']
smr = pd.merge(smr, JOB_m, on='UID', how='left')
smr = pd.merge(smr, JOB_vnum, on='UID', how='left')
smr['TTL'] = smr['BP'] + smr['GR'] + smr['PM']
smr['TTL_uni'] = smr['TTL'] / (smr['GR_vnum'] + smr['BP_vnum'] + smr['PM_vnum'])
smr['GR_uni'] = smr['GR'] / smr['GR_vnum']
smr['BP_uni'] = smr['BP'] / smr['BP_vnum']
smr['PM_uni'] = smr['PM'] / smr['PM_vnum']
smr['GR_freq'] = smr['GR_vnum'] / smr['keika_max']
smr['BP_freq'] = smr['BP_vnum'] / smr['keika_max']
smr['PM_freq'] = smr['PM_vnum'] / smr['keika_max']
smr = pd.merge(smr, TB, on='UID', how='left')
smr = pd.merge(smr, PM, on='UID', how='left')
smr = pd.merge(smr, SA_cont, on='UID', how='left')

# Backup smr to check (can be removed)
smr_backup = smr.copy()
print(smr.head())

del cb
del JOB_m
del JOB_vnum
del TB
del PM

          UID      RUN   Total_VALUE  visit_num  keika_max  bad_q  START_year  \
0         0.0   2651.0  2.650011e+09          3        7.0      0        2016   
1  15768909.0   1000.0  5.763379e+09          3        1.0      0        2022   
2  70209999.0  13765.0  9.350061e+09          5        1.0      0        2022   
3  76716756.0   1059.0  3.937247e+09          3        1.0      0        2022   
4  90492647.0  10428.0  2.175882e+10          5        3.0      1        2019   

  START_date  total_buy  visit_freq_y  ...   BP_freq   PM_freq  TIRE  BATTERY  \
0 2016-02-20          3      0.428571  ...  0.142857  0.142857     0        0   
1 2022-10-13          1      3.000000  ...  1.000000  1.000000     0        0   
2 2022-04-26          1      5.000000  ...  0.000000  2.000000     0        0   
3 2022-05-23          1      3.000000  ...  0.000000  0.000000     0        0   
4 2019-11-30          1      1.666667  ...  0.333333  0.666667     0        0   

   OIL  latest_visit_year 

In [43]:
#smr.to_csv(output_folder + '/smr.csv')

In [44]:
# Maintanance Pack
mp = pd.read_csv(tmp1_folder + '/mp.csv')
mp['UID_count'] = mp['UID'].copy()
mp = mp.groupby('UID').agg({'UID_count': 'count'}).reset_index()
mp.columns = ['UID', 'mp_num']
mp['mp'] = 1
print(mp.head())

           UID  mp_num  mp
0   76716756.0       1   1
1   96871171.0       1   1
2  106172217.0       1   1
3  106842461.0       1   1
4  109008610.0       1   1


In [45]:
# Read tmp2 files calculated from code 3
MH = pd.read_csv(tmp2_folder + '/MH_new_name.csv')

chat = pd.read_csv(tmp2_folder + '/chat_output.csv')

CR = pd.read_csv(tmp2_folder + '/comp_output.csv')

#dcsi
dcsi = pd.read_csv(tmp2_folder + '/DCSI_output_ver2.csv')
dcsi = dcsi[dcsi['Year'] == dcsi.groupby('UID')['Year'].transform('max')]
dcsi.columns = ['UID', 'Month', 'Year', 'Dealer', 'Region', 'satisfaction_reminder', 'satisfaction_reception', 'satisfaction_customer_lounge', 'satisfaction_delivery', 'satisfaction_repair_quality', 'satisfaction_facility', 'one_time_repair', 'total_satisfaction', 'Free_comment', 'follow_call']
dcsi = dcsi.groupby('UID').agg({'total_satisfaction': 'mean', 'satisfaction_reminder': 'mean', 'satisfaction_reception': 'mean', 'satisfaction_customer_lounge': 'mean', 'satisfaction_delivery': 'mean', 'satisfaction_repair_quality': 'mean', 'satisfaction_facility': 'mean', 'one_time_repair': 'mean', 'follow_call': 'mean'}).reset_index()

ew = pd.read_csv(tmp2_folder + '/ew_output.csv')
ew = ew.drop('ew_all_car', axis=1)

eve_v_pro = pd.read_csv(tmp2_folder + '/event_v_promo_new_name.csv')
#eve_v_pro = eve_v_pro.drop('event_v_promo', axis=1)

book = pd.read_csv(tmp2_folder + '/book_new_name.csv')
book = book.drop(['UID_counts_book', 'UID有無'], axis=1)

test_drive = pd.read_csv(tmp2_folder + '/test_drive_output.csv')

estimate_cost = pd.read_csv(tmp2_folder + '/estimate_cost_output.csv')
estimate_cost.columns = ['UID', 'estimate_cost']

view_product_color = pd.read_csv(tmp2_folder + '/view_product_color_output.csv')
view_product_color.columns = ['UID', 'view_product_color']

view_product_detail = pd.read_csv(tmp2_folder + '/view_product_detail_am_output.csv')
view_product_detail.columns = ['UID', 'view_product_detail']

view_product_gallery = pd.read_csv(tmp2_folder + '/view_product_gallery_output.csv')
view_product_gallery.columns = ['UID', 'view_product_gallery']

view_product_list = pd.read_csv(tmp2_folder + '/view_product_list_output.csv')
view_product_list.columns = ['UID', 'view_product_list']

view_promotion_notification = pd.read_csv(tmp2_folder + '/view_promotion_notification_output.csv')

view_test_drive = pd.read_csv(tmp2_folder + '/view_test_drive_output.csv')

sca = pd.read_csv(tmp2_folder + '/SCA_output.csv')

notif_book_pi = pd.read_csv(tmp2_folder + '/notif_book_pi_new_name.csv')

notif_book_pm = pd.read_csv(tmp2_folder + '/notif_book_pm_new_name.csv')

notif_book_spa = pd.read_csv(tmp2_folder + '/notif_book_spa_new_name.csv')

notif_v_pro = pd.read_csv(tmp2_folder + '/notif_v_pro_new_name.csv')

notif_v_spa = pd.read_csv(tmp2_folder + '/notif_v_spa_new_name.csv')

notif_vpi = pd.read_csv(tmp2_folder + '/notif_vpi_new_name.csv')

notif_vpm = pd.read_csv(tmp2_folder + '/notif_vpm_new_name.csv')

print("All files read")

All files read


In [46]:
mp['UID'] = mp['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
MH['UID'] = MH['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
chat['UID'] = chat['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
CR['UID'] = CR['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
dcsi['UID'] = dcsi['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
ew['UID'] = ew['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
eve_v_pro['UID'] = eve_v_pro['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
book['UID'] = book['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
test_drive['UID'] = test_drive['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
estimate_cost['UID'] = estimate_cost['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
view_product_color['UID'] = view_product_color['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
view_product_detail['UID'] = view_product_detail['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
view_product_gallery['UID'] = view_product_gallery['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
view_product_list['UID'] = view_product_list['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
view_promotion_notification['UID'] = view_promotion_notification['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
view_test_drive['UID'] = view_test_drive['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
sca['UID'] = sca['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
notif_book_pi['UID'] = notif_book_pi['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
notif_book_pm['UID'] = notif_book_pm['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
notif_book_spa['UID'] = notif_book_spa['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
notif_v_pro['UID'] = notif_v_pro['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
notif_v_spa['UID'] = notif_v_spa['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
notif_vpi['UID'] = notif_vpi['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
notif_vpm['UID'] = notif_vpm['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
notif_vpm['UID'] = notif_vpm['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
tgt['UID'] = tgt['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
print("Cleaned UID fields for data")

Cleaned UID fields for data


In [47]:
# Load smr backup to test
smr = smr_backup.copy()
smr['UID'] = smr['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
print(smr.head())

        UID      RUN   Total_VALUE  visit_num  keika_max  bad_q  START_year  \
0         0   2651.0  2.650011e+09          3        7.0      0        2016   
1  15768909   1000.0  5.763379e+09          3        1.0      0        2022   
2  70209999  13765.0  9.350061e+09          5        1.0      0        2022   
3  76716756   1059.0  3.937247e+09          3        1.0      0        2022   
4  90492647  10428.0  2.175882e+10          5        3.0      1        2019   

  START_date  total_buy  visit_freq_y  ...   BP_freq   PM_freq  TIRE  BATTERY  \
0 2016-02-20          3      0.428571  ...  0.142857  0.142857     0        0   
1 2022-10-13          1      3.000000  ...  1.000000  1.000000     0        0   
2 2022-04-26          1      5.000000  ...  0.000000  2.000000     0        0   
3 2022-05-23          1      3.000000  ...  0.000000  0.000000     0        0   
4 2019-11-30          1      1.666667  ...  0.333333  0.666667     0        0   

   OIL  latest_visit_year  visit_year 

In [48]:
# Merge smr with other file
smr = pd.merge(smr, mp, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("mp", len(smr)))
smr = pd.merge(smr, MH, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("MH", len(smr)))
smr = pd.merge(smr, chat, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("chat", len(smr)))
smr = pd.merge(smr, CR, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("CR", len(smr)))
smr = pd.merge(smr, dcsi, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("dcsi", len(smr)))
smr = pd.merge(smr, ew, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("ew", len(smr)))
smr = pd.merge(smr, eve_v_pro, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("eve_v_pro", len(smr)))
smr = pd.merge(smr, book, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("book", len(smr)))
smr = pd.merge(smr, test_drive, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("test_drive", len(smr)))
smr = pd.merge(smr, estimate_cost, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("estimate_cost", len(smr)))
smr = pd.merge(smr, view_product_color, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("view_product_color", len(smr)))
smr = pd.merge(smr, view_product_detail, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("view_product_detail", len(smr)))
smr = pd.merge(smr, view_product_gallery, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("view_product_gallery", len(smr)))
smr = pd.merge(smr, view_product_list, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("view_product_list", len(smr)))
smr = pd.merge(smr, view_promotion_notification, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("view_promotion_notification", len(smr)))
smr = pd.merge(smr, view_test_drive, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("view_test_drive", len(smr)))
smr = pd.merge(smr, sca, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("sca", len(smr)))
smr = pd.merge(smr, notif_book_pi, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("notif_book_pi", len(smr)))
smr = pd.merge(smr, notif_book_pm, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("notif_book_pm", len(smr)))
smr = pd.merge(smr, notif_book_spa, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("notif_book_spa", len(smr)))
smr = pd.merge(smr, notif_v_pro, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("notif_v_pro", len(smr)))
smr = pd.merge(smr, notif_v_spa, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("notif_v_spa", len(smr)))
smr = pd.merge(smr, notif_vpi, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("notif_vpi", len(smr)))
smr = pd.merge(smr, notif_vpm, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("notif_vpm", len(smr)))
smr = pd.merge(smr, tgt, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("tgt", len(smr)))


print("Data merged")

Merged with: mp  - column count: 126525
Merged with: MH  - column count: 126525
Merged with: chat  - column count: 126525
Merged with: CR  - column count: 126525
Merged with: dcsi  - column count: 126525
Merged with: ew  - column count: 126525
Merged with: eve_v_pro  - column count: 126525
Merged with: book  - column count: 126525
Merged with: test_drive  - column count: 126525
Merged with: estimate_cost  - column count: 126525
Merged with: view_product_color  - column count: 126525
Merged with: view_product_detail  - column count: 126525
Merged with: view_product_gallery  - column count: 126525
Merged with: view_product_list  - column count: 126525
Merged with: view_promotion_notification  - column count: 126525
Merged with: view_test_drive  - column count: 126525
Merged with: sca  - column count: 126525
Merged with: notif_book_pi  - column count: 126525
Merged with: notif_book_pm  - column count: 126525
Merged with: notif_book_spa  - column count: 126525
Merged with: notif_v_pro  - c

In [49]:
smr = smr.assign(visit_rate=lambda x: x['visit_rate'].apply(lambda y: 100 if y > 1 else y*100))
smr = smr.assign(keika_max=lambda x: 2021-x['START_year'])
smr = smr[smr['keika_max'] > 0]
#smr = smr.drop(columns=['PM_freq', 'BP_freq', 'GR_freq', 'visit_year'])
#smr.drop(columns = ['CUSTOMER_ADVISER', 'visit_num', 'visit_freq_y', 'GR_vnum', 'PM_vnum', 'BP_vnum', 'TTL_uni', 'TTL', 'GR_uni', 'BP_uni', 'PM_uni', 'RUN', 'OIL', 'Total_VALUE', 'GR', 'PM', 'BP', 'ew_sales', 'app_mente_SA_select', 'app_mente_SA_select', 'CR_feedback', 'satisfaction_mechanic', 'mp_num', 'bad_q', 'notif_book_pi', 'notif_book_pm', 'notif_book_spa', 'test_drive', 'follow_call', 'one_time_repair', 'coupon_view', 'view_product_color', 'view_product_gallery', 'chat_num', 'sca_visit_num', 'view_product_list', 'view_promotion_notification', 'MH_Active', 'latest_visit_year', 'service_part'], axis=1, inplace=True)

drop_list = [col for col in smr.columns if col.startswith('satisfaction')]
drop_list.extend(['PM_freq', 'BP_freq', 'GR_freq', 'visit_year'])
drop_list.extend(['CUSTOMER_ADVISER', 'visit_num', 'visit_freq_y', 'GR_vnum', 'PM_vnum', 'BP_vnum', 'TTL_uni', 'TTL', 'GR_uni', 'BP_uni', 'PM_uni', 'RUN', 'OIL', 'Total_VALUE', 'GR', 'PM', 'BP', 'ew_sales', 'app_mente_SA_select', 'app_mente_SA_select', 'CR_feedback', 'satisfaction_mechanic', 'mp_num', 'bad_q', 'notif_book_pi', 'notif_book_pm', 'notif_book_spa', 'test_drive', 'follow_call', 'one_time_repair', 'coupon_view', 'view_product_color', 'view_product_gallery', 'chat_num', 'sca_visit_num', 'view_product_list', 'view_promotion_notification', 'MH_Active', 'latest_visit_year', 'service_part'])
drop_list.extend([col for col in smr.columns if col.startswith('UID有無')])
cols_to_drop = set(smr.columns).intersection(drop_list)

#smr = smr.drop(columns=cols_to_drop)
del mp, MH, chat, CR, dcsi, ew, eve_v_pro, book, test_drive, estimate_cost, view_product_color, view_product_detail, view_product_gallery, view_product_list, view_promotion_notification, view_test_drive, sca, notif_book_pi, notif_book_pm, notif_book_spa, notif_v_pro, notif_v_spa, notif_vpi, notif_vpm
print(smr.head())

          UID      RUN   Total_VALUE  visit_num  keika_max  bad_q  START_year  \
0           0   2651.0  2.650011e+09          3          5      0        2016   
4    90492647  10428.0  2.175882e+10          5          2      1        2019   
5    98359185     23.0  1.167576e+09          2          1      0        2020   
6   108059030  21255.0  6.826858e+09          5          1      0        2020   
10  112259319  35979.0  3.106026e+10         11          2      1        2019   

   START_date  total_buy  visit_freq_y  ...  view_test_drive  sca_visit_num  \
0  2016-02-20          3      0.428571  ...              NaN            NaN   
4  2019-11-30          1      1.666667  ...              NaN            NaN   
5  2020-11-30          1      2.000000  ...              NaN            NaN   
6  2020-09-29          1      2.500000  ...              NaN            NaN   
10 2019-04-23          1      2.750000  ...              NaN            NaN   

    notif_book_pi  notif_book_pm  noti

In [54]:
# Export merged data
data = smr[smr['keika_max'] > 0]
field_backup = data[['UID', 'START_date', 'START_year']]
data = data.drop(['UID', 'START_date', 'START_year'], axis=1)
data = data.apply(pd.to_numeric).fillna(0)
data = pd.concat([field_backup, data], axis=1)
data = data[data['UID'] != 0]
data = data[data['UID'] != '0']
#data = data.drop(['START_date', 'event_v_promo'], axis=1)
#data = data.drop(['START_year', 'UID'], axis=1)
output_data_folder = output_folder + "/dataset_all_full_columns.csv"
data.to_csv(output_data_folder, index=False)
print("Export data to", output_data_folder)
print(data.columns)
data_backup = data.copy()

Export data to D:\Software\temp\CeDX\data/output/dataset_all_full_columns.csv
Index(['UID', 'START_date', 'START_year', 'RUN', 'Total_VALUE', 'visit_num',
       'keika_max', 'bad_q', 'total_buy', 'visit_freq_y', 'GR', 'PM', 'BP',
       'GR_vnum', 'PM_vnum', 'BP_vnum', 'TTL', 'TTL_uni', 'GR_uni', 'BP_uni',
       'PM_uni', 'GR_freq', 'BP_freq', 'PM_freq', 'TIRE', 'BATTERY', 'OIL',
       'latest_visit_year', 'visit_year', 'visit_rate', 'CUSTOMER_ADVISER',
       'SA_cont', 'mp_num', 'mp', 'MH_Active', 'satisfaction_service',
       'satisfaction_mechanic', 'satisfaction_chatbot',
       'satisfaction_chatbot_agent', 'MH_ID', 'chat_num', 'service_part',
       'chat', 'CR_feedback', 'CR', 'total_satisfaction',
       'satisfaction_reminder', 'satisfaction_reception',
       'satisfaction_customer_lounge', 'satisfaction_delivery',
       'satisfaction_repair_quality', 'satisfaction_facility',
       'one_time_repair', 'follow_call', 'ew_sales', 'ew_num', 'event_v_promo',
       'coupon_

In [57]:
# Only keep relevant columns
data_shrink = data_backup.copy()
data_shrink = data_shrink[['UID', 'keika_max', 'total_buy', 'TIRE', 'BATTERY', 'visit_rate', 'SA_cont', 'mp', 'MH_ID', 'chat', 'CR', 'total_satisfaction', 'ew_num', 'coupon_use', 'SA_select', 'app_reserve_mean', 'estimate_cost', 'view_product_detail', 'view_test_drive', 'notif_v_spa', 'notif_vpi', 'notif_vpm', 'PM2ov']]
output_data_folder = output_folder + "/dataset_all.csv"
data_shrink.to_csv(output_data_folder, index=False)
print("Export data to", output_data_folder)
print(data_shrink.columns)
data_shrink_backup = data_shrink.copy()

Export data to D:\Software\temp\CeDX\data/output/dataset_all.csv
Index(['UID', 'keika_max', 'total_buy', 'TIRE', 'BATTERY', 'visit_rate',
       'SA_cont', 'mp', 'MH_ID', 'chat', 'CR', 'total_satisfaction', 'ew_num',
       'coupon_use', 'SA_select', 'app_reserve_mean', 'estimate_cost',
       'view_product_detail', 'view_test_drive', 'notif_v_spa', 'notif_vpi',
       'notif_vpm', 'PM2ov'],
      dtype='object')


# Apply ML to create model

In [58]:
data = data_shrink_backup.copy()
PM2ov = data['PM2ov']
data = data.drop('PM2ov', axis=1)
print(pd.Series(PM2ov).value_counts())
print(data.shape)
print(data.head())

0.0    42697
1.0    38640
Name: PM2ov, dtype: int64
(81337, 22)
          UID  keika_max  total_buy  TIRE  BATTERY  visit_rate  SA_cont   mp  \
4    90492647          2          1     0        0   50.000000      0.0  0.0   
5    98359185          1          1     0        0    0.000000      0.0  0.0   
6   108059030          1          1     0        1   66.666667      0.0  0.0   
10  112259319          2          1     0        1  100.000000      1.0  1.0   
12  123225861          3          1     0        1   60.000000      0.0  0.0   

    MH_ID  chat  ...  ew_num  coupon_use  SA_select  app_reserve_mean  \
4     0.0   0.0  ...     0.0         0.0        1.0               1.0   
5     0.0   0.0  ...     0.0         0.0        0.0               0.0   
6     0.0   0.0  ...     0.0         0.0        0.0               0.0   
10    0.0   1.0  ...     1.0         0.0        0.0               0.0   
12    0.0   1.0  ...     0.0         0.0        1.0               2.0   

    estimate_cos

In [64]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

data = data_shrink_backup.copy()
scaler = StandardScaler()
sc = scaler.fit_transform(data)
z = pd.DataFrame(sc, index=data.index, columns=data.columns)
output_data_folder = output_folder + "/dataset_all_scaled.csv"
z.to_csv(output_data_folder, index=False)
print("Export data to", output_data_folder)


Export data to D:\Software\temp\CeDX\data/output/dataset_all_scaled.csv


In [65]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

data = data_shrink_backup.copy()
#data = data.drop('UID', axis=1)
scaler = StandardScaler()
sc = scaler.fit_transform(data)

z = pd.DataFrame(sc, index=data.index, columns=data.columns)
z['PM2ov'] = PM2ov
X = z.drop('PM2ov', axis=1)
y = z['PM2ov']

print(z)
formula = "PM2ov~keika_max + total_buy + TIRE + BATTERY + visit_rate + mp + MH_ID + chat + CR + total_satisfaction + ew_num + coupon_use + SA_select + app_reserve_mean + estimate_cost + view_product_detail + view_test_drive + notif_v_spa + notif_vpi + notif_vpm + SA_cont + visit_rate:keika_max+SA_select:SA_cont"
#formula = "PM2ov ~ keika_max + TIRE + BATTERY + visit_rate + mp + chat + CR + ew_num + coupon_use + SA_select + app_reserve_mean + notif_vpm + SA_cont + keika_max:visit_rate"
model = sm.formula.glm(formula = formula, data = z, family = sm.families.Binomial()).fit()
best_model = model
while True:
    pvalues = best_model.pvalues.drop('Intercept')
    if pvalues.max() > 0.05:
        worst_feature = pvalues.idxmax()
        print("Removed %s" % (worst_feature))
        new_formula = formula.replace(f" + {worst_feature}", "")
        #new_formula = new_formula.replace(f" {worst_feature}:", "")
        #new_formula = new_formula.replace(f":{worst_feature} ", "")
        new_model = sm.formula.glm(formula=new_formula, data=data, family=sm.families.Binomial()).fit()
        if new_model.aic < best_model.aic:
            best_model = new_model
            formula = new_formula
        else:
            break
    else:
        break
# View the final model summary
print(model.aic)
print(formula)
print(model.summary())
data_out = data_shrink_backup.copy()
data_out['Score'] = model.fittedvalues
output_data_folder = output_folder + "/dataset_all_scored.csv"
data_out.to_csv(output_data_folder, index=False)


             UID  keika_max  total_buy      TIRE   BATTERY  visit_rate  \
4      -5.650972  -0.395099  -0.127401 -0.269743 -0.620792   -0.894343   
5      -5.595997  -1.248311  -0.127401 -0.269743 -0.620792   -2.819964   
6      -5.528210  -1.248311  -0.127401 -0.269743  0.725411   -0.252469   
10     -5.498856  -0.395099  -0.127401 -0.269743  0.725411    1.031279   
12     -5.422217   0.458112  -0.127401 -0.269743  0.725411   -0.509218   
...          ...        ...        ...       ...       ...         ...   
126502  0.689992   2.164534  -0.127401 -0.269743 -0.620792   -2.269786   
126503  0.690122  -0.395099  -0.127401 -0.269743 -0.620792   -0.894343   
126504  0.690497   0.458112  -0.127401 -0.269743  0.725411   -0.509218   
126505  0.697366  -0.395099  -0.127401 -0.269743 -0.620792    0.068468   
126506  0.697527   0.458112  -0.127401 -0.269743 -0.620792    0.261030   

         SA_cont        mp     MH_ID      chat  ...  coupon_use  SA_select  \
4      -0.559473 -0.378387 -0.364

## Only run code below to test HM's dataset_all

In [61]:
# Save model after creating it
model.save('customer_engagement_index_model.pkl')